In [49]:
import pandas as pd
import xgboost
import shap
import RDG_interventional_classification
import RDG_gaussian_classification
import numpy as np
import random
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.model_selection import train_test_split

In [50]:
from sktime.classification.kernel_based import RocketClassifier
from sktime.datasets import load_gunpoint
from sktime.datasets import load_basic_motions
from sktime.datasets import load_italy_power_demand

X, y = load_gunpoint()


numpy_X = []
for i in range(0,X.shape[0]):
    numpy_X.append(X.loc[i].values[0])
X = np.asarray(numpy_X)

y = np.asarray(y)

X = (X - np.min(X))/(np.max(X)-np.min(X))
#y =  (y - np.min(y))/(np.max(y)-np.min(y))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

In [51]:
X.shape

(200, 150)

In [52]:
feature_names = np.arange(100)

In [53]:
y

array(['2', '2', '1', '1', '2', '2', '2', '2', '2', '1', '1', '1', '1',
       '1', '2', '1', '2', '2', '1', '2', '1', '1', '1', '2', '1', '2',
       '1', '1', '2', '1', '1', '2', '2', '1', '2', '1', '2', '2', '2',
       '2', '2', '1', '1', '1', '2', '2', '1', '2', '1', '2', '1', '2',
       '2', '1', '1', '2', '1', '2', '2', '1', '1', '1', '2', '1', '1',
       '1', '1', '2', '2', '2', '1', '2', '1', '1', '1', '2', '1', '1',
       '2', '1', '1', '2', '2', '1', '1', '1', '1', '1', '1', '2', '1',
       '2', '1', '1', '2', '1', '2', '2', '1', '2', '2', '2', '1', '2',
       '1', '1', '2', '2', '1', '2', '1', '2', '1', '1', '2', '1', '1',
       '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '1',
       '1', '1', '2', '2', '2', '2', '1', '2', '1', '1', '1', '2', '1',
       '2', '1', '1', '2', '2', '1', '1', '2', '1', '1', '2', '2', '1',
       '1', '2', '1', '2', '2', '2', '1', '2', '1', '1', '2', '1', '1',
       '2', '1', '2', '2', '2', '1', '2', '1', '1', '2', '2', '2

In [54]:
reg = RocketClassifier(num_kernels=100)
reg.fit(np.asarray(X_train), np.asarray(y_train))
clf = reg
clf.predict_proba(X_test[3].reshape(1,100))[0][0]

ValueError: cannot reshape array of size 150 into shape (1,100)

In [118]:
from sktime.classification.deep_learning.cnn import CNNClassifier
cnn = CNNClassifier(n_epochs=30,batch_size=4,verbose=False)  
cnn.fit(X_train, y_train)
cnn.predict_proba(X_test[0].reshape(1,-1))[0][0]
cnn.score(X_test,y_test)

2023-03-17 09:54:24.806573: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


17/17 [==============================] - 0s 3ms/step


2023-03-17 09:54:30.070429: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


0.8181818181818182

In [119]:
clf = cnn

In [120]:

cov = np.cov(X.T)
means = X.mean(axis=0)

minis = np.min(X,axis=0)
maxis = np.max(X,axis=0)

In [105]:
cov.shape

(150, 150)

ground_truth = {}
for i in range(0,len(clf.coef_[0])):
    ground_truth[i] = np.abs(clf.coef_[0][i])
sorted_indexes = sorted(ground_truth, key=ground_truth.get,reverse=True)
sorted_indexes

In [141]:
import timeit
import random

starttime = timeit.default_timer()
print("The start time is :",starttime)
e = shap.KernelExplainer(clf.predict_proba,X[0:10])
ks_shap_values = e.shap_values(X[0:1])[0]
print("The time difference is :", timeit.default_timer() - starttime)

The start time is : 455612.215524708
3/3 [==============================] - 0s 4ms/step


  0%|          | 0/1 [00:00<?, ?it/s]

5870/5870 [==============================] - 10s 2ms/step


The default of 'normalize' will be set to False in version 1.2 and deprecated in version 1.4.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LassoLarsIC())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * np.sqrt(n_samples). 
The default of 'normalize' will be set to False in version 1.2 and deprecated in version 1.4.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LassoLarsIC())

If you wish to pass a sample_weight

The time difference is : 12.006321042019408


In [144]:
12 * 200

2400

In [136]:
39.36 * 10

393.6

In [145]:
starttime = timeit.default_timer()
print("The start time is :",starttime)
x = X_test[0]
cond_seps,cond_nonseps = RDG_gaussian_classification.RDG_separate(clf.predict_proba,x,means,cov,minis,maxis,0.5,X_train)
#int_seps,int_nonseps = RDG_interventional_classification.RDG_separate(clf.predict_proba,x,means,minis,maxis,0.2)
print("The time difference is :", timeit.default_timer() - starttime)
starttime = timeit.default_timer()
print("The start time is :",starttime)
for a in range(0,1):
    #ss_i_atts= RDG_interventional_classification.atts_separate(clf.predict_proba,x,means,minis,maxis,0.2,int_seps,int_nonseps)
    ss_g_atts = RDG_gaussian_classification.atts_separate(clf.predict_proba,x,means,cov,minis,maxis,0.2,X_train,cond_seps,cond_nonseps)
print("The time difference is :", timeit.default_timer() - starttime)

The start time is : 455740.767166625
1/1 [==============================] - 0s 9ms/step


1/1 [==============================] - 0s 8ms/step


1/1 [==============================] - 0s 9ms/step


1/1 [==============================] - 0s 9ms/step
The time difference is : 15.034260915999766
The start time is : 455755.801471541
34/34 [==============================] - 0s 2ms/step


1/1 [==============================] - 0s 8ms/step


34/34 [==============================] - 0s 2ms/step
The time difference is : 17.147829541994724


In [146]:
17 * 200

3400

In [133]:

starttime = timeit.default_timer()
print("The start time is :",starttime)
cond_seps,cond_nonseps = RDG_gaussian_classification.RDG_separate(clf.predict_proba,x,means,cov,minis,maxis,0.0002,X)
int_seps,int_nonseps = RDG_interventional_classification.RDG_separate(clf.predict_proba,x,means,minis,maxis,0.0002)

"""print(cond_seps)
print("The time difference is :", timeit.default_timer() - starttime)

starttime = timeit.default_timer()
print("The start time is :",starttime)
for a in range(0,200):
    x = X[a]
    print(a)
    #ss_i_atts= RDG_interventional_classification.atts_separate(clf.predict_proba,x,means,minis,maxis,0.2,int_seps,int_nonseps)
    ss_g_atts = RDG_gaussian_classification.atts_separate(clf.predict_proba,x,means,cov,minis,maxis,0.2,X,cond_seps,cond_nonseps)
print("The time difference is :", timeit.default_timer() - starttime)"""
  

The start time is : 451149.877661541
1/1 [==============================] - 0s 9ms/step


1/1 [==============================] - 0s 9ms/step


1/1 [==============================] - 0s 9ms/step


1/1 [==============================] - 0s 8ms/step


1/1 [==============================] - 0s 8ms/step


...

1/1 [==============================] - 0s 8ms/step


1/1 [==============================] - 0s 9ms/step


1/1 [==============================] - 0s 8ms/step


1/1 [==============================] - 0s 7ms/step


'print(cond_seps)\nprint("The time difference is :", timeit.default_timer() - starttime)\n\nstarttime = timeit.default_timer()\nprint("The start time is :",starttime)\nfor a in range(0,200):\n    x = X[a]\n    print(a)\n    #ss_i_atts= RDG_interventional_classification.atts_separate(clf.predict_proba,x,means,minis,maxis,0.2,int_seps,int_nonseps)\n    ss_g_atts = RDG_gaussian_classification.atts_separate(clf.predict_proba,x,means,cov,minis,maxis,0.2,X,cond_seps,cond_nonseps)\nprint("The time difference is :", timeit.default_timer() - starttime)'

In [134]:
print(int_nonseps)

[[12, 33, 35, 52, 15, 16, 17, 18, 19, 20, 22, 23, 26, 30, 31, 32, 34, 36, 37, 38, 39, 41, 42, 43, 44], [14, 29, 21, 24, 25, 27, 28, 91, 98, 100, 103, 135, 48, 49, 50, 51, 53, 54, 57, 88, 89, 96, 97, 99, 105, 107, 108, 109, 111, 112, 124, 125, 126, 127, 129, 130], [134, 143]]


In [135]:
56*200 + 13

11213

In [ ]:
ks_shap_values[0][0]

In [ ]:
clf.predict_proba(X_test)

In [137]:
overall_ss_g_sum_indexes = []
overall_ss_i_sum_indexes = []
overall_ks_sum_indexes = []

overall_tree_sum_indexes = []


overall_ss_g_indexes = []
overall_ss_i_indexes = []
overall_ks_indexes = []

overall_tree_indexes = []

ss_g_scores = []
ss_i_scores = []
ks_scores = []
og_scores = []
#tree_scores = []

x = X_test[0]
#cond_seps,cond_nonseps = RDG_gaussian_classification.RDG_separate(clf.predict_proba,x,means,cov,minis,maxis,0.0004,X_train)
#int_seps,int_nonseps = RDG_interventional_classification.RDG_separate(clf.predict_proba,x,means,minis,maxis,0.0004)

print(int_nonseps)
print(cond_nonseps)
for a in range(0,20):
    x = X[a]
    ss_i_atts= RDG_interventional_classification.atts_separate(clf.predict_proba,x,means,minis,maxis,0.2,int_seps,int_nonseps)
    ss_g_atts = RDG_gaussian_classification.atts_separate(clf.predict_proba,x,means,cov,minis,maxis,0.2,X_train,cond_seps,cond_nonseps)
    
    ks_atts = {}
    for val in range(0,len(ks_shap_values[a])):
        ks_atts[val] = ks_shap_values[a][val]
    
        
    #tree_atts = {}
    #for val in range(0,len(tree_shap_values[a])):
        #tree_atts[val] = tree_shap_values[a][val]

    print(ks_atts)
    temp = {}
    for u in ss_i_atts.keys():
        temp[u] = np.abs(ss_i_atts[u])
    sorted_ss_i_indexes = sorted(temp, key=temp.get, reverse=True)
    
    temp = {}
    for u in ss_g_atts.keys():
        temp[u] = np.abs(ss_g_atts[u]) 
    sorted_ss_g_indexes = sorted(temp, key=temp.get, reverse=True)
    
    temp = {}
    for u in list(ks_atts.keys()):
        temp[u] = np.abs(ks_atts[u]) 
    print(temp)
    sorted_ks_indexes = sorted(temp, key=temp.get, reverse=True)
    
    
    #temp = {}
    #for u in tree_atts.keys():
       # temp[u] = np.abs(tree_atts[u]) 
    #sorted_tree_indexes = sorted(temp, key=temp.get, reverse=True)
    
    overall_ss_g_indexes.append(sorted_ss_g_indexes[0])
    overall_ss_i_indexes.append(sorted_ss_i_indexes[0])
    overall_ks_indexes.append(sorted_ks_indexes[0])
    #overall_og_indexes.append(sorted_og_indexes[0])
    #overall_tree_indexes.append(sorted_tree_indexes[0])
    
    total_sum = clf.predict_proba(x.reshape(1,-1))[0][0] -  clf.predict_proba(means.reshape(1,-1))[0][0]

    overall_ss_i_sum_indexes.append(sum(ss_i_atts.values()) - total_sum)
    overall_ss_g_sum_indexes.append(sum(ss_g_atts.values()) - total_sum)
    overall_ks_sum_indexes.append(sum(ks_atts.values()) - total_sum)
    #overall_og_sum_indexes.append(sum(og_atts.values()) - total_sum)
    
    
    ind = sorted_ss_g_indexes[0]
    new_x = x.copy()
    if len(list(ind)) > 1:
        for u in list(ind):
            new_x[u] = means[u]
    else:
        new_x[list(ind)[0]] = means[list(ind)[0]]
    
    prediction = clf.predict_proba(new_x.reshape(1,-1))[0][0]
    
    ss_g_scores.append(np.abs(prediction - clf.predict_proba(means.reshape(1,-1))[0][0]))
    
    ind = sorted_ss_i_indexes[0]
    new_x = x.copy()
    if len(list(ind)) > 1:
        for u in list(ind):
            new_x[u] = means[u]
    else:
        new_x[list(ind)[0]] = means[list(ind)[0]]
    
    prediction = clf.predict_proba(new_x.reshape(1,-1))[0][0]
    
    ss_i_scores.append(np.abs(prediction - clf.predict_proba(means.reshape(1,-1))[0][0]))
    
    ind = sorted_ks_indexes[0]
    new_x = x.copy()
    
    new_x[ind] = means[ind]
    
    prediction = clf.predict_proba(new_x.reshape(1,-1))[0][0]
    
    ks_scores.append(np.abs(prediction - clf.predict_proba(means.reshape(1,-1))[0][0]))
    
    

    
    """ind = sorted_tree_indexes[0]
    new_x = x.copy()
    
    new_x[ind] = means[ind]
    
    prediction = clf.predict(new_x.reshape(1,-1))
    
    tree_scores.append(np.abs(prediction - clf.predict(means.reshape(1,-1))))"""
            

[[12, 33, 35, 52, 15, 16, 17, 18, 19, 20, 22, 23, 26, 30, 31, 32, 34, 36, 37, 38, 39, 41, 42, 43, 44], [14, 29, 21, 24, 25, 27, 28, 91, 98, 100, 103, 135, 48, 49, 50, 51, 53, 54, 57, 88, 89, 96, 97, 99, 105, 107, 108, 109, 111, 112, 124, 125, 126, 127, 129, 130], [134, 143]]
[[10, 33, 34, 35, 19, 20, 21, 22, 25, 26, 29, 30, 31, 32, 36, 37, 39, 40, 41, 43, 44, 45, 46, 99, 100, 101, 102, 103, 105, 106, 107, 108, 109, 110, 112, 124, 125, 129, 130, 132, 136, 144, 147, 11, 12, 13, 14, 15, 16, 17, 18, 23, 24, 27, 28, 38, 42, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 72, 93, 94, 95, 96, 97, 98, 104, 114, 115, 116, 118, 119, 121, 122, 131, 133, 145, 146, 148, 61, 62, 63, 64, 65, 66, 92, 113, 126, 67, 68, 71, 73, 75, 76, 77, 117, 120, 123, 139, 140, 141, 91, 111, 127, 128, 134, 135, 69, 70, 87, 88, 89]]
1/1 [==============================] - 0s 9ms/step


34/34 [==============================] - 0s 2ms/step
in here

34/34 [==============================] - 0s 2ms/step
{0: 0.000827383793569933, 1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0012586533152701973, 5: 0.0, 6: 0.0009600946619168077, 7: 0.0023318480268005315, 8: 0.0015650678434511733, 9: 0.003833021656182281, 10: 0.004836158429203763, 11: 0.006481232519895214, 12: 0.006864562854470801, 13: 0.008543644021345803, 14: 0.011506255661313233, 15: 0.012787055485043303, 16: 0.012751680589546038, 17: 0.012506503832752036, 18: 0.013200262920082098, 19: 0.009363585491562597, 20: 0.008373949325239993, 21: 0.0063177924847793165, 22: 0.0030956173564301026, 23: 0.0007305285442142512, 24: 0.0008513201014319557, 25: 0.0007454171588143627, 26: 0.0015393336693737616, 27: 0.004486762230450317, 28: 0.009638208856223698, 29: 0.014657497070820244, 30: 0.02136640803585023, 31: 0.028474017420121393, 32: 0.036195274209364026, 33: 0.039996504695317694, 34: 0.04455077774501135, 35: 0.046471133017686336, 36: 0.046813029703

1/1 [==============================] - 0s 8ms/step


34/34 [==============================] - 0s 2ms/step
in here

34/34 [==============================] - 0s 2ms/step
{0: 0.0, 1: 0.0, 2: 0.0, 3: -0.00016305412134638708, 4: 0.00040699807300470514, 5: 0.0, 6: 0.001119245333479596, 7: 0.0018956817741136583, 8: 0.002440584716068561, 9: 0.0038024357847362827, 10: 0.005149501122391126, 11: 0.006858524045696278, 12: 0.00848110202050803, 13: 0.010037918854966868, 14: 0.012160451621298016, 15: 0.01430200688409583, 16: 0.014467637297474986, 17: 0.014539042743405589, 18: 0.014563402899215573, 19: 0.012547992122078848, 20: 0.010392888293771385, 21: 0.007631735366840468, 22: 0.004329562685429639, 23: 0.001776635753642175, 24: 0.0, 25: -0.0004912547029779571, 26: 0.0, 27: 0.0025948136005691236, 28: 0.006603018380844067, 29: 0.011603476879248176, 30: 0.017568767543692493, 31: 0.02339484323619155, 32: 0.02959376672850654, 33: 0.03501455889509347, 34: 0.03943793283576849, 35: 0.04016621097391839, 36: 0.0407294894388181, 37: 0.03746557663402163, 38: 0.03

1/1 [==============================] - 0s 7ms/step


...

1/1 [==============================] - 0s 7ms/step


34/34 [==============================] - 0s 2ms/step
in here

34/34 [==============================] - 0s 2ms/step
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.00023859356703750045, 6: 0.0, 7: 0.0, 8: 0.0, 9: 0.0, 10: 0.0, 11: 0.0, 12: 0.0, 13: 0.0, 14: 0.0, 15: -0.005347105179372429, 16: 0.0, 17: 0.0, 18: 0.0, 19: 0.0, 20: 0.0, 21: 0.0, 22: 0.0, 23: 0.0, 24: 0.0, 25: 0.0006880166889938618, 26: 0.0030393061968165738, 27: 0.006515780078052031, 28: 0.010062019488945015, 29: 0.01421391947144002, 30: 0.01924085998573875, 31: 0.024207074091940384, 32: 0.02892646439343822, 33: 0.03226817680535301, 34: 0.03245543781079161, 35: 0.030645283272670968, 36: 0.02223731668923146, 37: 0.014456452448035145, 38: 0.005228131848145046, 39: 0.0, 40: 0.0, 41: 0.0003758208396511828, 42: 0.0035158547149081033, 43: 0.006105637002476089, 44: 0.010492334418156012, 45: 0.013203359279719006, 46: 0.015307969552912167, 47: 0.01625888482200048, 48: 0.013262144181833009, 49: 0.013018866607619908, 50: 0.0111017018627

1/1 [==============================] - 0s 8ms/step


34/34 [==============================] - 0s 2ms/step
in here

34/34 [==============================] - 0s 2ms/step
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0049695066199973445, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0, 8: -0.0018293267858376793, 9: -0.004753072327220737, 10: -0.0022439011169040736, 11: 0.0, 12: -0.004755393023311605, 13: 0.0, 14: 0.0, 15: -0.005595112685959847, 16: -0.008525452590628329, 17: -0.004407762138589193, 18: -0.0017111371080554688, 19: -0.0037719432358422024, 20: -0.004737246877316512, 21: 0.0, 22: 0.0, 23: 0.008165292080945349, 24: 0.0, 25: -0.0043577405023861905, 26: -0.01169897395830926, 27: -0.025290364525151324, 28: -0.035954111810032115, 29: -0.04012346260069452, 30: -0.050249529350874116, 31: -0.053342293703719575, 32: -0.056861217543725945, 33: -0.05923568075250155, 34: -0.058552349767757285, 35: -0.05554371415745631, 36: -0.050254257584616796, 37: -0.03689528153708027, 38: -0.02634911564136166, 39: -0.014943422190508893, 40: -0.006760095699617619, 41: 0.00403719666619099

1/1 [==============================] - 0s 8ms/step


In [131]:
int_nonseps

[[11,
  17,
  27,
  33,
  34,
  13,
  14,
  16,
  18,
  19,
  20,
  22,
  24,
  25,
  26,
  28,
  29,
  30,
  31,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  46,
  47,
  51,
  52,
  55,
  56,
  63,
  72,
  73,
  88,
  91,
  92,
  93,
  109,
  110,
  116,
  119,
  143,
  12,
  15,
  21,
  23,
  32,
  35,
  45,
  48,
  49,
  50,
  53,
  66,
  67,
  68,
  82,
  85,
  90,
  94,
  95,
  97,
  98,
  99,
  100,
  101,
  103,
  106,
  107,
  108,
  111,
  114,
  115,
  118,
  120,
  121,
  122,
  123,
  125,
  134,
  135,
  136,
  137,
  140,
  54,
  57,
  58,
  64,
  65,
  84,
  86,
  87,
  89,
  102,
  104,
  105,
  113,
  117,
  124,
  126,
  127,
  128,
  130,
  131,
  132,
  133,
  139,
  141,
  62,
  71,
  96,
  112,
  138,
  59,
  70,
  146]]

In [132]:
cond_nonseps

[[4,
  41,
  42,
  43,
  44,
  45,
  46,
  7,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  47,
  48,
  49,
  50,
  51,
  95,
  96,
  97,
  98,
  99,
  100,
  101,
  102,
  103,
  104,
  105,
  106,
  107,
  108,
  109,
  110,
  111,
  112,
  113,
  114,
  124,
  129,
  130,
  5,
  6,
  8,
  22,
  23,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  60,
  62,
  63,
  64,
  65,
  66,
  70,
  71,
  72,
  73,
  88,
  90,
  91,
  92,
  93,
  94,
  115,
  116,
  117,
  118,
  119,
  121,
  122,
  125,
  128,
  131,
  135,
  136,
  137,
  140,
  143,
  144,
  147,
  148,
  77,
  89,
  120,
  126,
  142,
  145,
  59,
  67,
  68,
  69,
  74,
  83,
  123,
  133,
  139,
  141,
  146,
  149,
  61,
  75,
  76,
  86,
  127,
  134,
  138,
  78,
  84,
  85,
  132,
  82,
  87]]

In [138]:
print('interventional')
print('sum')
print(np.mean(overall_ss_i_sum_indexes))
print(np.std(overall_ss_i_sum_indexes))

print('faithfulness')
print(np.mean(ss_i_scores))
print(np.std(ss_i_scores))

print('gaussian conditional')

print('sum')
print(np.mean(overall_ss_g_sum_indexes))
print(np.std(overall_ss_g_sum_indexes))

print('faithfulness')
print(np.mean(ss_g_scores))
print(np.std(ss_g_scores))

print('KS')

print('sum')
print(np.mean(overall_ks_sum_indexes))
print(np.std(overall_ks_sum_indexes))

print('faithfulness')
print(np.mean(ks_scores))
print(np.std(ks_scores))


print(" ")
print("ATTS:")

print("")

print("KS")

print(ks_atts)

print("I")

print(ss_i_atts)

print("G")

print(ss_g_atts)



interventional
sum
0.08611363768577576
0.12974182429541087
faithfulness
0.2501069
0.09269557
gaussian conditional
sum
0.24066412448883057
0.007205838141354596
faithfulness
0.010793209
0.005113946
KS
sum
-5.587935447692871e-10
1.7768502261805267e-09
faithfulness
0.28032202
0.14081419
 
ATTS:

KS
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0049695066199973445, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0, 8: -0.0018293267858376793, 9: -0.004753072327220737, 10: -0.0022439011169040736, 11: 0.0, 12: -0.004755393023311605, 13: 0.0, 14: 0.0, 15: -0.005595112685959847, 16: -0.008525452590628329, 17: -0.004407762138589193, 18: -0.0017111371080554688, 19: -0.0037719432358422024, 20: -0.004737246877316512, 21: 0.0, 22: 0.0, 23: 0.008165292080945349, 24: 0.0, 25: -0.0043577405023861905, 26: -0.01169897395830926, 27: -0.025290364525151324, 28: -0.035954111810032115, 29: -0.04012346260069452, 30: -0.050249529350874116, 31: -0.053342293703719575, 32: -0.056861217543725945, 33: -0.05923568075250155, 34: -0.058552349767757285, 3

In [ ]:
np.mean(clf.predict_proba(X))

In [ ]:
print(overall_ss_g_indexes)
print(overall_ss_i_indexes)
print(overall_ks_indexes)

In [ ]:
selected_x = x.copy()

In [ ]:
selected_x

In [ ]:
fig, ax = plt.subplots()
x = np.arange(0, 24)

ax.plot(x, selected_x, color='black')
ax.plot(x, means, color='blue')

#threshold = 0.1
#ax.axhline(threshold, color='green', lw=2, alpha=0.7)
ax.axvspan(18, 23, alpha=0.5, color='red')

ax.axvspan(0, 4, alpha=0.5, color='green')

ax.axvspan(12, 16, alpha=0.5, color='green')
ax.set_ylabel('Power Demand')
ax.set_xlabel('Hour of Day')
plt.axvline(x=18,color='black')
plt.savefig('time_series_2.pdf')

In [ ]:
clf.predict_proba(X_test[100].reshape(1,-1))

In [ ]:
clf.predict_proba(means.reshape(1,-1))